In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset
from tqdm.auto import tqdm
from functools import partial
from einops import rearrange
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from baukit import TraceDict

# Download the modelz
device = "cuda:0"
model_name="EleutherAI/Pythia-70M-deduped"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# set seed
torch.manual_seed(0)
np.random.seed(0)

/root/logan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from autoencoders import *
# ae_model_id = ["jbrinkma/Pythia-70M-chess_sp51_r4_gpt_neox.layers.1", "jbrinkma/Pythia-70M-chess_sp51_r4_gpt_neox.layers.2.mlp"]
model_id = "jbrinkma/Pythia-70M-deduped-SAEs"
autoencoders = []
layers = model.config.num_hidden_layers
cache_names = [(f"gpt_neox.layers.{i}", f"gpt_neox.layers.{i+1}.mlp") for i in range(layers-1)]
# cache_names = [(f"gpt_neox.layers.{i}", f"gpt_neox.layers.{i+1}.attention") for i in range(layers-1)]
# cache_names = [(f"gpt_neox.layers.{i}", f"gpt_neox.layers.{i+1}.mlp") for i in range(1,2)]
num_layers = len(cache_names)
cache_names = [item for sublist in cache_names for item in sublist]
filenames = [(f"Pythia-70M-deduped-{i}.pt", f"Pythia-70M-deduped-mlp-{i+1}.pt") for i in range(layers-1)]
# filenames = [(f"Pythia-70M-deduped-{i}.pt", f"Pythia-70M-deduped-attention-{i+1}.pt") for i in range(layers-1)]
# filenames = [(f"Pythia-70M-deduped-{i}.pt", f"Pythia-70M-deduped-mlp-{i+1}.pt") for i in range(1,2)]
filenames = [item for sublist in filenames for item in sublist]
for filen in filenames:
    ae_download_location = hf_hub_download(repo_id=model_id, filename=filen)
    autoencoder = torch.load(ae_download_location)
    autoencoder.to_device(device)
    # Freeze autoencoder weights
    autoencoder.encoder.requires_grad_(False)
    autoencoder.encoder_bias.requires_grad_(False)
    autoencoders.append(autoencoder)

In [6]:
from activation_dataset import chunk_and_tokenize
# Download the dataset
# This formats it, so every datapoint is max_length tokens long
# The batch size is for loading activations from the LLM, not for inference on the autoencoder
dataset_name="stas/openwebtext-10k"
max_seq_length=256
dataset = load_dataset(dataset_name, split="train")
dataset, _ = chunk_and_tokenize(dataset, tokenizer, max_length=max_seq_length)
max_tokens = dataset.num_rows*max_seq_length
print(f"Number of tokens: {max_tokens/1e6:.2f}M")

Found cached dataset openwebtext-10k (/root/.cache/huggingface/datasets/stas___openwebtext-10k/plain_text/1.0.0/3a8df094c671b4cb63ed0b41f40fb3bd855e9ce2e3765e5df50abcdfb5ec144b)
Loading cached processed dataset at /root/.cache/huggingface/datasets/stas___openwebtext-10k/plain_text/1.0.0/3a8df094c671b4cb63ed0b41f40fb3bd855e9ce2e3765e5df50abcdfb5ec144b/cache-89f7d956ed9de0f6_*_of_00008.arrow


Number of tokens: 11.23M


In [7]:
# split dataset into both a train & test set
test_size = 0.1
test_size = int(len(dataset)*test_size)
train_size = len(dataset)-test_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print(f"Number of Train Tokens: {len(train_dataset)*max_seq_length/1e6:.2f}M")
print(f"Number of Test Tokens: {len(test_dataset)*max_seq_length/1e6:.2f}M")

Number of Train Tokens: 10.10M
Number of Test Tokens: 1.12M


In [8]:
# Create dataloaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [9]:
from torch.optim import Adam
from torch import nn
    
class mlp(nn.Module):
    def __init__(self, input_size, output_size=None, hidden_size=None, bias=True):
        super().__init__()
        if(output_size is None):
            output_size = input_size
        if(hidden_size is None):
            hidden_size = input_size
        # Only adding a bias to the latent layer
        self.linear = nn.Linear(input_size, hidden_size, bias=bias)
        self.linear2 = nn.Linear(hidden_size, output_size, bias=False)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x
    def latent(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x
    
class linear(nn.Module):
    def __init__(self, input_size, output_size=None, bias=True):
        super().__init__()
        if(output_size is None):
            output_size = input_size
        self.linear = nn.Linear(input_size, output_size, bias=bias)
    def forward(self, x):
        x = self.linear(x)
        return x

In [10]:
# Load in the features indices
# Note: linear & MLP features are a subet of alive features, not total features
import pickle
with open("linear_features.pkl", "rb") as f:
    linear_features = pickle.load(f)
with open("mlp_features.pkl", "rb") as f:
    mlp_features = pickle.load(f)
with open("alive_features.pkl", "rb") as f:
    alive_features_ind = pickle.load(f)
normalization_per_layer = torch.load("normalization_per_layer.pt")

In [11]:
num_features, d_model = autoencoder.encoder.shape

mlp_weights = []
mlp_optimizers = []
for auto_ind in range(num_layers):
    mlp_weights.append(mlp(num_features, bias=False))
    mlp_weights[-1].to(device)
    mlp_optimizers.append(Adam(mlp_weights[-1].parameters(), lr=1e-3))

In [24]:
for i, batch in enumerate(tqdm(test_dataloader)):
    batch = batch["input_ids"].to(device)
    with torch.no_grad():
        with TraceDict(model, cache_names) as ret:
            original_logits = model(batch)
    break

  0%|          | 0/138 [00:01<?, ?it/s]


In [35]:
logits = original_logits.logits

# Assuming logits and batch are provided as follows
# logits.shape = (batch_size, seq, vocab_size)
# batch.shape = (batch_size, seq)

# Reshape logits and labels for CrossEntropyLoss
logits = logits.view(-1, logits.size(-1))  # Reshape to (batch_size * seq, vocab_size)
labels = batch.view(-1)  # Flatten to (batch_size * seq)

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Calculate loss
loss = loss_fn(logits, labels)

# Print or return the loss
print(loss)

tensor(12.4122, device='cuda:0')


torch.Size([804864, 512])

In [30]:
batch.shape

torch.Size([32, 256])

In [15]:
# Find the arguments that model() takes in (ie def of model.forward())
arguments = model.forward.__code__.co_varnames
arguments

('self',
 'input_ids',
 'attention_mask',
 'position_ids',
 'inputs_embeds',
 'head_mask',
 'past_key_values',
 'labels',
 'use_cache',
 'output_attentions',
 'output_hidden_states',
 'return_dict',
 'outputs',
 'hidden_states',
 'lm_logits',
 'lm_loss',
 'shift_logits',
 'loss_fct',
 'output')

In [20]:
def x(a = 1, b = 2, c = 3):
    return a+b+c
x.__code__.co_varnames

('a', 'b', 'c')

In [41]:
from baukit import TraceDict, Trace
def replace_with_reconstructed_features(mlp_out, layer_name):
    # We will have the true activations at MLP_out, & the reconstruction. 
    return x_hat

sparse_weights = mlp_weights
ce = nn.CrossEntropyLoss()

original_loss = [0]*num_layers
reconstructed_loss = [0]*num_layers
loss_diff = [0]*num_layers
# Cross entropy check
for i, batch in enumerate(tqdm(test_dataloader)):
    batch = batch["input_ids"].to(device)
    with torch.no_grad():
        with TraceDict(model, cache_names) as ret:
            original_logits = model(batch).logits
        for cache_name_ind in range(len(cache_names) // 2):
            sparse_weight = sparse_weights[cache_name_ind]
            input_cache_name = cache_names[cache_name_ind * 2]
            input_autoencoder = autoencoders[cache_name_ind * 2]
            input_activations = ret[input_cache_name].output
            if isinstance(input_activations, tuple):
                input_activations = input_activations[0]
            input_internal_activations = rearrange(input_activations, "b s n -> (b s) n")
            input_internal_activations = input_autoencoder.encode(input_internal_activations)
            x_hat = sparse_weights[cache_name_ind](input_internal_activations)
            # Decode x_hat
            output_autoencoder = autoencoders[cache_name_ind * 2 + 1]
            x_hat = output_autoencoder.decode(x_hat)
            # Rearrange x_hat back to b s n
            x_hat = rearrange(x_hat, "(b s) n -> b s n", b=batch.shape[0], s=batch.shape[1])
            # replace the activations with the reconstructed activations
            output_cache_name = cache_names[cache_name_ind * 2 + 1]
            with Trace(model, output_cache_name, edit_output=replace_with_reconstructed_features) as ret_2:
                logits = model(batch).logits
            logits = logits.view(-1, logits.size(-1))  # Reshape to (batch_size * seq, vocab_size)
            original_logits = original_logits.view(-1, original_logits.size(-1))  # Reshape to (batch_size * seq, vocab_size)
            labels = batch.view(-1)  # Flatten to (batch_size * seq)

            # Define the loss function
            ce = nn.CrossEntropyLoss()
            # Calculate loss
            recon_loss = ce(logits, labels)
            orig_loss = ce(original_logits, labels)
            loss_diff[cache_name_ind] += recon_loss - orig_loss
            reconstructed_loss[cache_name_ind] += recon_loss
            original_loss[cache_name_ind] += orig_loss
            divide_by = i + 1
            print(f"Layer:{cache_name_ind+1} | Original Loss: {original_loss[cache_name_ind]/divide_by:.2f} | Reconstructed Loss: {reconstructed_loss[cache_name_ind]/divide_by:.2f} | Difference: {loss_diff[cache_name_ind]/divide_by:.2f}")
# Average
reconstructed_loss /= len(dataset)
original_loss /= len(dataset)
loss_diff /= len(dataset)
print(f"Original Loss: {original_loss:.2f}")
print(f"Reconstructed Loss: {reconstructed_loss:.2f}")
# Find percentage of loss is reconstructed
print(f"Percentage of loss reconstructed: {100*(original_loss-reconstructed_loss)/original_loss:.2f}%")

  0%|          | 0/138 [00:00<?, ?it/s]

Layer:1 | Original Loss: 10.93 | Reconstructed Loss: 9.30 | Difference: -1.63
Layer:2 | Original Loss: 10.93 | Reconstructed Loss: 9.73 | Difference: -1.20
Layer:3 | Original Loss: 10.93 | Reconstructed Loss: 10.80 | Difference: -0.13
Layer:4 | Original Loss: 10.93 | Reconstructed Loss: 9.44 | Difference: -1.48


  1%|          | 1/138 [00:03<07:33,  3.31s/it]

Layer:5 | Original Loss: 10.93 | Reconstructed Loss: 16.37 | Difference: 5.44
Layer:1 | Original Loss: 10.86 | Reconstructed Loss: 9.34 | Difference: -1.52
Layer:2 | Original Loss: 10.86 | Reconstructed Loss: 9.69 | Difference: -1.17
Layer:3 | Original Loss: 10.86 | Reconstructed Loss: 10.68 | Difference: -0.18
Layer:4 | Original Loss: 10.86 | Reconstructed Loss: 9.44 | Difference: -1.42


  1%|▏         | 2/138 [00:05<05:25,  2.39s/it]

...

 99%|█████████▊| 136/138 [04:08<00:03,  1.86s/it]

Layer:5 | Original Loss: 11.44 | Reconstructed Loss: 16.89 | Difference: 5.45
Layer:1 | Original Loss: 11.44 | Reconstructed Loss: 9.88 | Difference: -1.56
Layer:2 | Original Loss: 11.44 | Reconstructed Loss: 10.21 | Difference: -1.23
Layer:3 | Original Loss: 11.44 | Reconstructed Loss: 11.21 | Difference: -0.22
Layer:4 | Original Loss: 11.44 | Reconstructed Loss: 10.03 | Difference: -1.41


100%|██████████| 138/138 [04:10<00:00,  1.82s/it]

Layer:5 | Original Loss: 11.44 | Reconstructed Loss: 16.89 | Difference: 5.45
Layer:1 | Original Loss: 11.41 | Reconstructed Loss: 9.86 | Difference: -1.55
Layer:2 | Original Loss: 11.41 | Reconstructed Loss: 10.19 | Difference: -1.23
Layer:3 | Original Loss: 11.41 | Reconstructed Loss: 11.19 | Difference: -0.22
Layer:4 | Original Loss: 11.41 | Reconstructed Loss: 10.01 | Difference: -1.40
Layer:5 | Original Loss: 11.41 | Reconstructed Loss: 16.85 | Difference: 5.44


TypeError: unsupported operand type(s) for /=: 'list' and 'int'

In [50]:
[o_loss/(i+1) for o_loss in original_loss], [r_loss/(i+1) for r_loss in reconstructed_loss], [l_diff/(i+1) for l_diff in loss_diff]

([tensor(11.4106, device='cuda:0'),
  tensor(11.4106, device='cuda:0'),
  tensor(11.4106, device='cuda:0'),
  tensor(11.4106, device='cuda:0'),
  tensor(11.4106, device='cuda:0')],
 [tensor(9.8566, device='cuda:0'),
  tensor(10.1851, device='cuda:0'),
  tensor(11.1898, device='cuda:0'),
  tensor(10.0089, device='cuda:0'),
  tensor(16.8508, device='cuda:0')],
 [tensor(-1.5540, device='cuda:0'),
  tensor(-1.2255, device='cuda:0'),
  tensor(-0.2208, device='cuda:0'),
  tensor(-1.4017, device='cuda:0'),
  tensor(5.4402, device='cuda:0')])

In [ ]:


def train_sparse_model_and_check_CE(model, dataset, device, sparse_weights, autoencoders, cache_names, optimizers, test_datasest=None):
    mse = nn.MSELoss()
    losses = [[] for _ in range(len(sparse_weights))]

    for i, batch in enumerate(tqdm(dataset)):
        batch = batch["input_ids"].to(device)
        with torch.no_grad():
            with TraceDict(model, cache_names) as ret:
                _ = model(batch)
        
        for cache_name_ind in range(len(cache_names) // 2):
            sparse_weight = sparse_weights[cache_name_ind]

            input_cache_name = cache_names[cache_name_ind * 2]
            output_cache_name = cache_names[cache_name_ind * 2 + 1]
            input_autoencoder = autoencoders[cache_name_ind * 2]
            output_autoencoder = autoencoders[cache_name_ind * 2 + 1]
            input_activations = ret[input_cache_name].output
            output_activations = ret[output_cache_name].output

            if isinstance(input_activations, tuple):
                input_activations = input_activations[0]
            if isinstance(output_activations, tuple):
                output_activations = output_activations[0]

            input_internal_activations = rearrange(input_activations, "b s n -> (b s) n")
            output_internal_activations = rearrange(output_activations, "b s n -> (b s) n")

            input_internal_activations = input_autoencoder.encode(input_internal_activations)
            output_internal_activations = output_autoencoder.encode(output_internal_activations)

            x_hat = sparse_weights[cache_name_ind](input_internal_activations)

            # loss = mse(x_hat, output_internal_activations)
            current_alive_features_ind = alive_features_ind[cache_name_ind].nonzero(as_tuple=True)[0]
            indexed_x_hat = x_hat.index_select(dim=1, index=current_alive_features_ind.to(device))
            indexed_output_internal_activations = output_internal_activations.index_select(dim=1, index=current_alive_features_ind.to(device))
            loss = mse(indexed_x_hat, indexed_output_internal_activations)
            l1_loss = torch.norm(sparse_weight.linear.weight, dim=1, p=1).mean()
            # loss += l1_alpha * l1_loss

            loss.backward()
            optimizers[cache_name_ind].step()
            optimizers[cache_name_ind].zero_grad()
            losses[cache_name_ind].append(loss.item())

            if i % 100 == 0:
                print(f"Layers: {cache_name_ind} | MSE Loss: {loss.item()} | L1 Loss: {l1_loss.item()}")
            
        if i > 3000:
            # Stop training after 1000 batches
            break
    return losses

mlp_losses = train_sparse_model(model, dataset, device, mlp_weights, autoencoders, cache_names, mlp_optimizers)
